In [6]:
# testing lib fn
from __future__ import print_function, unicode_literals
from unicodedata import normalize
import library_py2 as lib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# from sklearn.metrics import roc
# from sklearn.model_selection import traintestsplit
# from sklearn.preprocessing import 
import json
import warnings
# from nltk.punkt impor
import nltk
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger')
import string
from nltk.stem.wordnet import WordNetLemmatizer, wordnet
from nltk.stem.porter import PorterStemmer
from nltk.collocations import *
from nltk import FreqDist
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
import re
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
warnings.filterwarnings(action='once')
%matplotlib inline
pd.set_option('display.max_columns', 999)
%load_ext autoreload
%autoreload 2

from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding, Masking
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import os
import argparse
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [109]:
df = pd.read_pickle('df1_pos_W.pkl')

In [110]:
df = lib.concat_word_pos(df, columns=['context_lemma_pos', 'question_lemma_pos','answers_lemma_pos'])

In [111]:
vocabulary = lib.get_total_vocab(df, columns=['context_lemma_pos', 'question_lemma_pos'])

In [133]:
df['text'] = df['context_lemma_pos'] + ' ' + df['question_lemma_pos']

In [136]:
df['question_answer'] = df['question_lemma_pos'] + ' ' + df['answers_lemma_pos']

In [86]:
df.head()

,context_lemma_pos,question_lemma_pos,answers_lemma_pos,answer_len,question_len,context_len
2,beyoncé_NN giselle_NN knowlescarter_NN biːˈjɒn...,when_WRB beyonce_NN leave_VBP destinys_VBN chi...,2003_CD,1,8,75
11,beyoncé_NN giselle_NN knowlescarter_NN biːˈjɒn...,when_WRB beyoncé_NN release_NN dangerously_RB ...,2003_CD,1,5,75
12,beyoncé_NN giselle_NN knowlescarter_NN biːˈjɒn...,how_WRB many_JJ grammy_JJ award_NNS beyoncé_NN...,five_CD,1,9,75
15,follow_VBG disbandment_NN destiny_NN child_NN ...,after_IN second_JJ solo_NN album_NN entertainm...,act_VBG,1,8,112
17,follow_VBG disbandment_NN destiny_NN child_NN ...,to_TO set_VB record_NN grammys_NN many_JJ beyo...,six_CD,1,7,112


In [145]:
train_data, valid_data, vocabulary, reversed_dictionary, vocabulary_size, embedding_matrix, embedding_dim = lib.load_sequential_data(df,
                                                                                                                                    pos_dict, 
                                                                                                                                    use_pos=True)

First five words  in vectorized format :  [119851, 33624, 47559, 1183, 106349]
Vocabulary examples : 
1.  booty_NNS 88384
2.  crescent_NN 19743
3.  congregatio_NN 69065
 Size of vocabulary :  127983
the_DT predominant_JJ educational_JJ psychology_NN 1750s_CD onward_NN especially_RB northern_JJ european_JJ country_NNS associationism_VBP notion_NN mind_NN associate_VBZ dissociates_NNS idea_NNS repeat_VBD routine_NNS in_IN addition_NN conducive_JJ enlightenment_NN ideology_NNS liberty_VBP selfdetermination_NN personal_JJ responsibility_NN offer_VBD practical_JJ theory_NN



In [146]:
embedding_dim

51

In [147]:
vocabulary_size

127983

In [150]:
len(train_data)

1027462

In [151]:
df.head()

,context_lemma_pos,question_lemma_pos,answers_lemma_pos,answer_len,question_len,context_len,text,question_answer
0,beyonc_NN giselle_NN knowlescarter_NN bijnse_N...,when_WRB beyonce_NN leave_VBP destinys_VBN chi...,2003_CD,1,8,75,beyonc_NN giselle_NN knowlescarter_NN bijnse_N...,when_WRB beyonce_NN leave_VBP destinys_VBN chi...
1,beyonc_NN giselle_NN knowlescarter_NN bijnse_N...,when_WRB beyonc_NN release_NN dangerously_RB l...,2003_CD,1,5,75,beyonc_NN giselle_NN knowlescarter_NN bijnse_N...,when_WRB beyonc_NN release_NN dangerously_RB l...
2,beyonc_NN giselle_NN knowlescarter_NN bijnse_N...,how_WRB many_JJ grammy_JJ award_NNS beyonc_NN ...,five_CD,1,9,75,beyonc_NN giselle_NN knowlescarter_NN bijnse_N...,how_WRB many_JJ grammy_JJ award_NNS beyonc_NN ...
3,follow_VBG disbandment_NN destiny_NN child_NN ...,after_IN second_JJ solo_NN album_NN entertainm...,act_VBG,1,8,112,follow_VBG disbandment_NN destiny_NN child_NN ...,after_IN second_JJ solo_NN album_NN entertainm...
4,follow_VBG disbandment_NN destiny_NN child_NN ...,to_TO set_VB record_NN grammys_NN many_JJ beyo...,six_CD,1,7,112,follow_VBG disbandment_NN destiny_NN child_NN ...,to_TO set_VB record_NN grammys_NN many_JJ beyo...


In [155]:
batch_size=10
num_epochs=20


In [157]:
train_data_generator = lib.KerasBatchGenerator(train_data, num_steps=df.question_len.max(),
                                              batch_size=batch_size,
                                              vocabulary=vocabulary, 
                                              skip_step=df.question_len.max()+1)
valid_data_generator = lib.KerasBatchGenerator(valid_data, num_steps=df.question_len.max(),
                                              batch_size=batch_size,
                                              vocabulary=vocabulary, 
                                              skip_step=df.question_len.max()+1)


In [158]:
df.question_len.max()

22

In [160]:
hidden_size = embedding_dim
use_dropout=True
num_epochs=20
num_steps = df.question_len.max()

model_seq = Sequential([
    layers.Embedding(input_dim=vocabulary_size, 
                     output_dim=hidden_size,input_length=num_steps,
                     weights = [embedding_matrix],
                     mask_zero=True, trainable=False),
    layers.LSTM(hidden_size, return_sequences=True),
    layers.LSTM(hidden_size, return_sequences=True),
    layers.LSTM(hidden_size, dropout=0.25, return_sequences=True),
    layers.TimeDistributed(Dense(vocabulary_size)),
    layers.Activation('softmax')
])


optimizer=Adam()
model_seq.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

print(model_seq.summary())

/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:560: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(tensor.tensor_content, dtype=dtype).reshape(shape)
/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 22, 51)            6527133   
_________________________________________________________________
lstm_1 (LSTM)                (None, 22, 51)            21012     
_________________________________________________________________
lstm_2 (LSTM)                (None, 22, 51)            21012     
_________________________________________________________________
lstm_3 (LSTM)                (None, 22, 51)            21012     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 22, 127983)        6655116   
_________________________________________________________________
activation_1 (Activation)    (None, 22, 127983)        0         
Total params: 13,245,285
Trainable params: 6,718,152
Non-trainable params: 6,527,133
_________________________________________________________

In [163]:
model_seq.fit_generator(train_data_generator.generate(one_sequence=True), steps_per_epoch = 20, 
                    epochs=num_epochs,
                    validation_data=valid_data_generator.generate(one_sequence=True),
                    validation_steps=20)

Epoch 1/20
20/20 [==============================] - 181s 9s/step - loss: 11.7279 - categorical_accuracy: 0.0148 - val_loss: 11.6059 - val_categorical_accuracy: 0.0161
Epoch 2/20
20/20 [==============================] - 190s 10s/step - loss: 11.3384 - categorical_accuracy: 0.0134 - val_loss: 10.9511 - val_categorical_accuracy: 0.0159
Epoch 3/20
 1/20 [>.............................] - ETA: 2:44 - loss: 11.0107 - categorical_accuracy: 0.0136

KeyboardInterrupt: 